In [21]:
# Hamiltonian Neural Networks | 2019
# Sam Greydanus, Misko Dzamba, Jason Yosinski

import torch, argparse
import numpy as np

import os, sys
THIS_DIR = os.path.dirname(os.path.abspath(""))
THIS_DIR = os.path.join(THIS_DIR, "whitened-Mol-HNN-cuda")
# PARENT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(PARENT_DIR)

from cuda_nn_models import MLP
from cuda_hnn import HNN
from cuda_utils import L2_loss, to_pickle, from_pickle
# from data import get_dataset, coords2state, get_orbit, random_config
# from data import potential_energy, kinetic_energy, total_energy

In [5]:
##
# Loading in the whitened Dataset
##
import glob


files = glob.glob('./../SMD_data/*.npy')

dataset = []

for file_ in files:
    X_positions = np.load(file_)
    X = X_positions

    # Sample down the amount of sequenced frames from 20K to 2K
#     X = X[::10]
    #print(X.shape)
    # Create Training dataset from this sequence
    dataset.append(X)

In [7]:
num_trajectories = 100
num_atoms = 40


dataset = np.array(dataset)
dataset = dataset.reshape(num_trajectories, -1, num_atoms*3)

In [3]:
##
# Loading in the datasets
##
# num_trajectories = 100
# num_atoms = 40

# dataset = []
# for i in range(num_trajectories):
#     NUM = str(i).zfill(2)
#     fName = "../../10_deca_alanine/" + NUM + "/backbone.npy"
#     rawCoord = np.load(fName)
#     dataset.append(rawCoord)

# dataset = np.array(dataset) ## shape: (100, 20000, 40, 3)
# dataset = dataset.reshape(num_trajectories, -1, num_atoms*3)

In [11]:
## Downsampling
# dataset = dataset[:, ::2, :] # downsampling just for training purposes to make it easier


In [9]:
x_dataset = []
dx_dataset = []

## Getting the velocity of each trajectory
for traj_idx in range(num_trajectories):
    traj = dataset[traj_idx]
#     traj = dataset[0]
    num_datapoints = traj.shape[0] - 1

    momenta = []
    for i in range(traj.shape[0] - 1):
        dx = traj[i+1] - traj[i]
        momenta.append(dx)

    momenta = np.array(momenta) ## convert to an np array

    ## stack the coordinates so that we have all the positions and all the momentums
    coords = np.stack((traj[:-1], momenta), 1) ## shape: (19999, 2, 120)
    coords = coords.reshape(num_datapoints, 2 * 120) ## shape: (19999, 240)
    
    ## Now we need to calculate the change in coords (positions and momentums)
    delta_coords = []
    for frame in range(coords.shape[0] - 1):
        dx = coords[frame+1] - coords[frame]
        delta_coords.append(dx)
    delta_coords = np.array(delta_coords)
    
    ## appending to the dataset
    x_dataset.append(coords[:-1])
    dx_dataset.append(delta_coords)


x_dataset = np.array(x_dataset) ## shape of (100, 19998, 240)
dx_dataset = np.array(dx_dataset) ## shape of (100, 19998, 240)

x_dataset = x_dataset.reshape(-1, 240) ## shape of (1999800, 240)
dx_dataset = dx_dataset.reshape(-1, 240) ## shape of (1999800, 240)

# x_dataset[0] + dx_dataset[0] == x_dataset[1] ## This is just a sanity check that we can predict the next step

In [14]:
batch_size = 100
epochs = 3
total_steps = int(x_dataset.shape[0] / 100 * epochs)
total_steps

60060

In [22]:
## just for quick load don't need this

# np.save("whitened_x_dataset.npy", x_dataset)
# np.save("whitened_dx_dataset.npy", dx_dataset)

x_dataset = np.load("whitened_x_dataset.npy")
dx_dataset = np.load("whitened_dx_dataset.npy")

x = torch.tensor(x_dataset, requires_grad=True, dtype=torch.float32).cuda()
dxdt = torch.Tensor(dx_dataset).cuda()

## Training the HNN

In [23]:
# arrange data
def get_args():
    return {'input_dim': 240, # 40 atoms, each with q_x, q_y, p_z, p_y
         'hidden_dim': 200,
         'learn_rate': 1e-3,
         'input_noise': 0.1, ## NO INPUT NOISE YET
         'batch_size': 100,
         'nonlinearity': 'leaky',
         'total_steps': total_steps, ## 3 epochs effectively i guess
         'field_type': 'helmholtz', ## change this? solenoidal
         'print_every': 200,
         'verbose': True,
         'name': '2body',
         'baseline' : False,
         'seed': 0,
         'save_dir': '{}'.format(THIS_DIR),
         'fig_dir': './figures'}

class ObjectView(object):
    def __init__(self, d): self.__dict__ = d

args = ObjectView(get_args())

In [24]:
output_dim = 2
nn_model = MLP(args.input_dim, args.hidden_dim, output_dim, args.nonlinearity)
model = HNN(args.input_dim, differentiable_model=nn_model,
        field_type=args.field_type, baseline=args.baseline)
optim = torch.optim.Adam(model.parameters(), args.learn_rate, weight_decay=0)

In [25]:
# vanilla train loop
stats = {'train_loss': [], 'test_loss': []}
for step in range(args.total_steps+1):

    # train step
    ixs = torch.randperm(x.shape[0])[:args.batch_size]
    dxdt_hat = model.time_derivative(x[ixs])
    dxdt_hat += args.input_noise * torch.randn(*x[ixs].shape).cuda() # add noise, maybe
    
    loss = L2_loss(dxdt[ixs], dxdt_hat)
    loss.backward()
    grad = torch.cat([p.grad.flatten() for p in model.parameters()]).clone()
    optim.step() ; optim.zero_grad()

    # run test data
#     test_ixs = torch.randperm(test_x.shape[0])[:args.batch_size]
#     test_dxdt_hat = model.time_derivative(test_x[test_ixs])
#     test_dxdt_hat += args.input_noise * torch.randn(*test_x[test_ixs].shape) # add noise, maybe
#     test_loss = L2_loss(test_dxdt[test_ixs], test_dxdt_hat)

    # logging
    stats['train_loss'].append(loss.item())
#     stats['test_loss'].append(test_loss.item())
    if args.verbose and step % args.print_every == 0:
        print("step {}, train_loss {:.4e}, test_loss {:.4e}, grad norm {:.4e}, grad std {:.4e}"
          .format(step, loss.item(), 420, grad@grad, grad.std()))

print("=> Finished Training <=")

## Saving model
PATH = "white-MOLHNNv1.pt"
torch.save(model.state_dict(), PATH)

step 0, train_loss 4.7467e-02, test_loss 4.2000e+02, grad norm 1.9623e-07, grad std 9.6804e-07


KeyboardInterrupt: 